In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1'
import tensorflow as tf

import time

from pddlvm.utilities import tf, tfp, np, plt, tfb, floatf, tffloat, tfint, updateTfFloat,  _0
updateTfFloat ( tf.float32 )

from pddlvm.poisson1DPDEState import PDE_State
from pddlvm.poisson1DSolver import Solver 
from pddlvm.NeuralNet import NeuralN
from pddlvm.VEModel import VE
tf.random.set_seed(42)

import time

print(tffloat(0.))
tf.config.list_physical_devices()

# This is the one used for PDDLVM paper

In [ ]:
'''Create PDEState Object with Cheby Num terms (-1 for degree)'''
omegaLength = 2.0
numFElems   = 100
numFENodes  = numFElems + 1
xFE = tffloat( tf.linspace( 0., omegaLength, numFElems + 1) ) - 1.

pdeState = PDE_State(xFE, dimK=4, dimF=1, dimU=8)
'''Modify pdeState Bijcetor for kappa'''
pdeState.kappaBij   = tfb.Chain( bijectors=[tfb.Shift(tffloat(1.)), tfb.Softplus()] )


In [ ]:
'''Create solver Object'''
solver = Solver( pdeState, xFE, isNonLinear=True )

'''Modify pdeState.stress-strain for nonlinear Youngs Modulus'''
import types
def stress_strain_nonLinear(self, index, kappa, u_trial):
    #print('custom stress-strain')
    dudx = ( u_trial[index] - u_trial[index+1]) / (self.pdeState.xFE[index] - self.pdeState.xFE[index+1])
    return (tf.keras.activations.sigmoid(tf.math.abs(dudx)) + kappa*5. ) /10.
solver.stress_strain = types.MethodType(stress_strain_nonLinear, solver)


In [ ]:
model = VE( pdeState, solver )

def createNNsPrior():
    dimKFBlBr = model.dimK + model.dimF + model.dimBl + model.dimBr
    nh = 50
    U_KFBlBr  = NeuralN('U_of_KFBlBr', 
                    np.array([dimKFBlBr,
                              nh, nh, nh, nh,
                              model.dimU])
                    )

    k  = model.dimK
    bl = k  + model.dimBl
    br = bl + model.dimBr
    Z_UF     = NeuralN('Z_of_UF',
                    np.array([pdeState.dimU + model.dimF, 
                                nh, nh, nh, nh, 
                                model.dimK + model.dimBl + model.dimBr]),
                    multOutputLayout = {'K': tf.range(0,k),
                                        'Bl':tf.range(k,bl),'Br':tf.range(bl,br)}
                    )

    model.initPriorNetworks(U_KFBlBr=U_KFBlBr, Z_UF=Z_UF)
    model.setModePrior()
    return 

    
USE_UNIFORM_PRIOR = False
if USE_UNIFORM_PRIOR == True:
    model.LatentKappaPrior   = tfp.distributions.Uniform(low=tffloat([-0.5]*pdeState.dimK), high=tffloat([0.5]*pdeState.dimK))#works
    model.LatentForcingPrior = tfp.distributions.Uniform(low=tffloat([5.]*pdeState.dimF), high=tffloat([5.]*pdeState.dimF) )

USE_NORMAL_PRIOR = True
if USE_NORMAL_PRIOR == True:
    model.LatentKappaPrior   = tfp.distributions.Normal(tffloat([0.]*pdeState.dimK), tffloat([0.5]*pdeState.dimK))
    model.LatentForcingPrior = tfp.distributions.Uniform(low=tffloat([1.]*pdeState.dimF), high=tffloat([5.]*pdeState.dimF) )#works

model.LatentBlPrior      = tfp.distributions.Uniform(low=tffloat([0.]), high=tffloat([0.0]))
model.LatentBrPrior      = tfp.distributions.Uniform(low=tffloat([0.]), high=tffloat([1.5]) )


print(model.LatentForcingPrior.sample())
print(model.LatentBlPrior.sample())
createNNsPrior()


In [ ]:
totalTimeNNSolving = 0.
totalTimeSolving   = 0. 

In [ ]:
totalTimeNNSolving = 0.
totalTimeSolving   = 0. 

def computeSolve_TrueVsNN(samplesK, samplesF, samplesBcl, samplesBcr, PLOT=True, SAVE=False):
    
    global totalTimeNNSolving
    global totalTimeSolving  
    
    startSolving = time.time()
    meanSamplesU, logVarSamplesU = model.priorModels['U_of_KFBlBr'].Map([samplesK, samplesF, samplesBcl, samplesBcr])
    endSolving = time.time()
    totalTimeNNSolving += endSolving - startSolving

    us = model.reparameterize(meanSamplesU, logVarSamplesU )

    residualsU = tf.constant(0., shape=[1])
    residualsK = tf.constant(0., shape=[1])
    residualsB = tf.constant(0., shape=[1])
    
    if PLOT == True:
        fig, ax = plt.subplots(nrows=1, ncols=5, sharex=True,
                                            figsize=(24, 4))
    
    for i in range(samplesK.shape[0]):

        pdeState.kappa   = samplesK[i,:] 
        pdeState.forcing = samplesF[i,:] 
        pdeState.bdLeft  = samplesBcl[i,0] 
        pdeState.bdRight = samplesBcr[i,0] 

        startSolving = time.time()
        u_true     = solver.solve_tf_bfgs(  )
        u_trueSpec = solver.solve_tf_FESpectral_bfgs( )
        u_trueSpec = pdeState.u_funXFE( u_trueSpec[:,None] )
        endSolving = time.time()
        totalTimeSolving += endSolving - startSolving

        meanU, diagVarU = pdeState.gaussChebyGaussCoeffs(meanSamplesU[i,:], tf.exp(logVarSamplesU[i,:]), xFE )
        std2U = 2.*tf.sqrt(diagVarU)
        
        startSolving = time.time()
        uField = pdeState.u_funXFE(us[i,:][:,None])
        endSolving = time.time()
        totalTimeNNSolving += endSolving - startSolving

        kappaField = pdeState.kappa_fun( xFE )

        U_true_coeffs = pdeState.getChebyCoeffs_fromPoints(xFE, uField, model.dimU - 1)

        ZCoeffReconstMean, ZCoeffReconstlogvar = model.priorModels['Z_of_UF'].Map([ U_true_coeffs[None,:], samplesF[i,:][None,:]])
        kappaCoeffReconstMean = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['K'], axis=1)
        kappaCoeffReconstlogVar  = tf.gather(ZCoeffReconstlogvar, model.priorModels['Z_of_UF'].multOutputLayout['K'], axis=1)
        kappaCoeffReconstMean =  tf.reshape(kappaCoeffReconstMean,[-1])
        kappaCoeffReconstVar  =  tf.exp(tf.reshape(kappaCoeffReconstlogVar,[-1]))
        meanK, diagCovK = pdeState.gaussChebyGaussCoeffs(kappaCoeffReconstMean,kappaCoeffReconstVar, xFE)
        meanKT, varKT = pdeState.unscentedGaussTransf(meanK, diagCovK, pdeState.kappaBij, k=2)
        std2KT = 2*tf.sqrt(varKT)
        print('std2KT = ', std2KT)

        BlReconst = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['Bl'], axis=1)
        BrReconst = tf.gather(ZCoeffReconstMean, model.priorModels['Z_of_UF'].multOutputLayout['Br'], axis=1)

        residualUi = ( tf.linalg.norm(  u_true - meanU ) / tf.linalg.norm( u_true ) )**2.
        residualKi = ( tf.linalg.norm( kappaField - meanKT ) / tf.linalg.norm( kappaField ) )**2.
        residualBi = ( tf.linalg.norm( ( BlReconst - samplesBcl[i,0] ) + ( samplesBcr[i,0] - BrReconst ) ) / tf.linalg.norm( samplesBcl[i,0] + samplesBcr[i,0] ) )**2.

        residualsU = tf.concat([residualsU, residualUi[None]], 0)
        residualsK = tf.concat([residualsK, residualKi[None]], 0)
        residualsB = tf.concat([residualsB, residualBi[None]], 0)

        
        if PLOT == True:
            ax[0].plot( xFE , u_true)
            ax[0].set_xlabel(r'$x$')
            ax[0].set_ylabel(r'$u(x)$')
            ax[0].set_title('FE Solver')
            
            ax[1].plot(xFE, meanU)
            ax[1].fill_between(xFE, meanU + std2U, meanU - std2U, alpha=0.2)
            ax[1].set_xlabel(r'$x$')
            ax[1].set_ylabel(r'$u(x) \pm 2\sigma$')
            ax[1].set_title('NN Solver')

            ax[2].plot( xFE, kappaField)
            ax[2].set_xlabel(r'$x$')
            ax[2].set_ylabel(r'$\kappa(x)$')
            ax[2].set_title(r'True $\kappa(x)$')
            
            ax[3].plot(xFE, meanKT)
            ax[3].fill_between(xFE, meanKT + std2KT, meanKT- std2KT, alpha=0.2)
            ax[3].set_xlabel(r'$x$')
            ax[3].set_ylabel(r'$\kappa(x) \pm 2\sigma$')
            ax[3].set_title(r'Reconstructed $\kappa(x)$')

            ax[4].plot( xFE , tf.squeeze(pdeState.forcing_funXFE()))
            ax[4].set_xlabel(r'$x$')
            ax[4].set_ylabel(r'$f(x)$')
            ax[4].set_title(r'Forcing')


    if PLOT == True:
        plt.tight_layout()
        if SAVE == True:
            plt.savefig('figs/OneDpoissonNONLinearInRangeSolves.pdf')
        plt.show()

    return residualsU, residualsK, residualsB

In [ ]:
numSample = 3
PLOT = True
sampleKGenerator = model.LatentKappaPrior
samplesK   = sampleKGenerator.sample(sample_shape=(numSample))

samplesF   = model.LatentForcingPrior.sample(sample_shape=(numSample))
samplesBcl = model.LatentBlPrior.sample(sample_shape=(numSample))
samplesBcr = model.LatentBrPrior.sample(sample_shape=(numSample))

totalTimeNNSolving = 0.
totalTimeSolving   = 0. 

residualsU, residualsK, residualsB = computeSolve_TrueVsNN(samplesK, samplesF, samplesBcl, samplesBcr, PLOT=PLOT, SAVE=False)

print( 'totalTimeSolving = ', totalTimeSolving)
print( 'totalTimeNNSolving = ', totalTimeNNSolving)

In [ ]:
eps =1e-2

AllEpsU = []
AllEpsK = []
AllEpsB = []

totalTimeTraining  = 0.

trainModels      = True


intervalELBOsave = 1000

PLOT = True


@tf.function
def train_step_prior(optimizer, iteration):
    with tf.GradientTape(persistent=True) as tape:
        loss = model.compute_nelbo_prior(iteration)
    for priorvars in model.priorModels['trainable_variables']:
        gradients    = tape.gradient(loss, priorvars)
        gradients, globNorm = tf.clip_by_global_norm(gradients, 1.0)
        tf.debugging.check_numerics(gradients[0], "GRAD NOT FINITE")
        optimizer.apply_gradients(zip(gradients, priorvars))
    return loss

if trainModels:
       
    '''Optimization sequence'''
    model.n_sgd      = 1

    num_iterations   = tfint(500_000)

    model.epsilon_r.assign( tffloat(eps) )

    lr = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-3, int(num_iterations/8), 0.5, staircase=True, name=None
    )
    optimizer = tf.keras.optimizers.Adam( lr )

    elboAll     = []
    startTraining = time.time()
    for epoch in tf.range(0, num_iterations ):

        start_time = time.time()
        loss = train_step_prior(optimizer, epoch)

        end_time = time.time()

        if epoch % intervalELBOsave == 0:
            #tf.print('epsi_r = ', model.epsilon_r)
            elboAll.append(-loss)
            print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
                .format(epoch, -loss, end_time - start_time))

    endTraining = time.time()
    totalTimeTraining += endTraining - startTraining
    print("DONE TRAINING FOR EPSI", eps)




In [ ]:
'''If you want to load Model Weights'''
saveModelWeights = False
loadModelWeights = False
dirname = 'weightsModel-100/'
if saveModelWeights:
    print('MODEL WEIGHTS SAVED')
    
    os.makedirs(os.path.dirname(dirname), exist_ok=True)
    model.priorModels['U_of_KFBlBr'].NN.save_weights(dirname+'U_of_KFBlBr_weights.h5')
    model.priorModels['Z_of_UF'].NN.save_weights(dirname+'Z_of_UF_weights.h5')
    
    model.priorModels['U_of_KFBlBr'].NN.compile() 
    model.priorModels['Z_of_UF'].NN.compile()
    model.priorModels['U_of_KFBlBr'].NN.save(dirname+'U_of_KFBlBr_weights.model')
    model.priorModels['Z_of_UF'].NN.save(dirname+'Z_of_UF_weights.model')

if loadModelWeights:
    print('LOADED WEIGHTS SAVED')
    dirname = 'weightsModel-100/'
    model.priorModels['U_of_KFBlBr'].NN = tf.keras.models.load_model(dirname+'U_of_KFBlBr_weights.model')
    model.priorModels['Z_of_UF'].NN = tf.keras.models.load_model(dirname+'Z_of_UF_weights.model')

    model.priorModels['U_of_KFBlBr'].NN.compile()
    model.priorModels['Z_of_UF'].NN.compile()
    # Check its architecture
    model.priorModels['U_of_KFBlBr'].NN.summary()
    model.priorModels['Z_of_UF'].NN.summary()


#print(elboAll)
if trainModels:
    elboAll =  np.array(elboAll)
    np.savetxt('ELBOValsModel', elboAll)



In [ ]:
print( 'totalTimeTraining = ', totalTimeTraining)
print( 'totalTimeSolving = ', totalTimeSolving)
print( 'totalTimeNNSolving = ', totalTimeNNSolving)

In [ ]:
numSample  = 3
samplesK   = model.LatentKappaPrior.sample(sample_shape=(numSample))
samplesF   = model.LatentForcingPrior.sample(sample_shape=(numSample))
samplesBcl = model.LatentBlPrior.sample(sample_shape=(numSample))
samplesBcr = model.LatentBrPrior.sample(sample_shape=(numSample))

totalTimeNNSolving = 0.
totalTimeSolving   = 0. 

residualsU, residualsK, residualsB = computeSolve_TrueVsNN(samplesK, samplesF, samplesBcl, samplesBcr, PLOT=PLOT, SAVE=False)

print( 'totalTimeSolving = ', totalTimeSolving)
print( 'totalTimeNNSolving = ', totalTimeNNSolving)

In [ ]:
plt.plot(range(elboAll.shape[0]),  elboAll )
scale = 1e3
plt.ylim(-scale,scale)
plt.show()

In [ ]:
with open('Times.txt', 'w') as the_file:
    the_file.write('totalTimeTraining = '  + str(totalTimeTraining) + 's'+ '\n' )
    the_file.write('totalTimeSolving = '  + str(totalTimeSolving) + 's'+  '\n' )
    the_file.write('totalTimeNNSolving = ' + str(totalTimeNNSolving) + 's'+  '\n' )

In [ ]:
dataYEpsi =tffloat(0.005)
def missingPhysics(u, x):
    #return u
    return u * (tf.math.sin(x*20.) / 20. + 1.) 
def addIndtNoise(u):
    return u + tf.random.normal(stddev=dataYEpsi, shape=u.shape, dtype=floatf)

def plotTrueSlnSamples(samplesK, samplesF, samplesBcl, samplesBcr):
    
    fig, (ax0, ax1, ax2, ax3, ax4) = plt.subplots(nrows=1, ncols=5, sharex=False,
                                        figsize=(20, 3))
    
    u_All = tf.constant(0., shape=[1, model.dimY], dtype=floatf)

    dirname = 'figs/nonlinMissPhysDatset'

    for i in range(samplesK.shape[0]):
        
        model.pdeState.forcing = tf.reshape(samplesF[i,:], [-1]) 
        model.pdeState.kappa   = tf.reshape(samplesK[i,:], [-1]) 
        model.pdeState.bdLeft  = tf.reshape(samplesBcl[i,:], []) 
        model.pdeState.bdRight = tf.reshape(samplesBcr[i,:], []) 

        u_true     = solver.solve_tf_bfgs(  )
        u_missPhys = missingPhysics(u_true, xFE)
        u_missPhysNoise = addIndtNoise(u_missPhys)
        u_All = tf.concat([u_All,u_missPhysNoise[None,:] ], 0)

        ax0.plot( pdeState.xFE, u_missPhys)
        ax1.plot(pdeState.xFE,u_missPhysNoise)
        ax2.plot(pdeState.xFE, u_true)
        ax3.plot(pdeState.xFE, u_missPhysNoise - u_true)

        kappaField = pdeState.kappa_fun(xFE)
        ax4.plot( xFE, kappaField )

    plt.tight_layout()
    fig.canvas.draw()
    extent = ax1.get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
    fig.savefig('{}Datay.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

    extent = ax2.get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
    fig.savefig('{}TrueSol.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

    extent = ax3.get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
    fig.savefig('{}TrueRho.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

    extent = ax4.get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
    fig.savefig('{}TrueKappa.pdf'.format(dirname), bbox_inches=extent.expanded(1.01, 1.01))

    plt.show()
    u_All = u_All[1:,:]
    return u_All

model.dimY = pdeState.xFE.shape[0]

numData = 50
print('model.dimK = ', model.dimK)
samplesKdata   = model.LatentKappaPrior.sample(sample_shape=(numData))
samplesFdata   = model.LatentForcingPrior.sample(sample_shape=(numData))
samplesBldata  = model.LatentBlPrior.sample(sample_shape=(numData))
samplesBrdata  = model.LatentBrPrior.sample(sample_shape=(numData))


priorFEpsi = tf.constant(1e-3,dtype=floatf)
priorMeanF = samplesFdata + tf.random.normal(stddev=priorFEpsi,  shape=samplesFdata.shape, dtype=floatf)
priorStdF  = tf.tile(priorFEpsi[None,None], priorMeanF.shape)
model.dataForcingPrior = (priorMeanF, tf.math.log(priorStdF)*2 )

variableSTD = tffloat(1e-1)

priorKEpsi = variableSTD
priorMeanK = samplesKdata 
priorStdK  = tf.tile(priorKEpsi[None,None], priorMeanK.shape)
model.dataKappaPrior = (priorMeanK,  tf.math.log(priorStdK)*2)

priorBlEpsi = variableSTD
priorMeanBl = samplesBldata 
priorStdBl  = tf.tile(priorBlEpsi[None,None], priorMeanBl.shape)
model.dataBlPrior = (priorMeanBl,  tf.math.log(priorStdBl)*2)

priorBrEpsi = variableSTD
priorMeanBr = samplesBrdata 
priorStdBr  = tf.tile(priorBrEpsi[None,None], priorMeanBr.shape)
model.dataBrPrior = (priorMeanBr,  tf.math.log(priorStdBr)*2)

model.dataY = plotTrueSlnSamples(samplesKdata, samplesFdata, samplesBldata, samplesBrdata)


In [ ]:
np.savetxt("{}ModelDataset".format(dirname), model.dataY)

In [ ]:
def createNNsPost():
    
    nh = 200 

    Y_U     = NeuralN('Y_of_U',
                np.array([model.dimU,
                            nh,
                            model.dimY]),
                )


    k  = model.dimK
    bl = k  + model.dimBl
    br = bl + model.dimBr
    dimKBlBr = model.dimK + model.dimBl + model.dimBr
    KBlBr_Y  = NeuralN('KBlBr_of_Y', 
                    np.array([model.dimY,
                              nh,
                              dimKBlBr]),
                        multOutputLayout = {'K': tf.range(0,k),
                                            'Bl':tf.range(k,bl),'Br':tf.range(bl,br)}
                    )


    model.initPostNetworks( Y_U=Y_U, KBlBr_Y=KBlBr_Y)
    model.setModePost()
    return 

createNNsPost()

In [ ]:

totalTimeTrainingPost  = 0.

trainModels      = True


intervalELBOsave = 1000

PLOT = False

model.epsilon_y.assign(tffloat(0.1)) 

model.setModePost()

timeStartTraingPost = time.time()

clipELBO = tfb.SoftClip(low=tffloat(-1000.), high=tffloat(1000.), hinge_softness=5.)
loss = model.compute_nelbo_post(0)
@tf.function
def train_step_post(optimizer, iteration):
    with tf.GradientTape(persistent=True) as tape:
        loss = model.compute_nelbo_post(iteration)
    for postvars in model.postModels['trainable_variables']:
        gradients = tape.gradient(loss, postvars)
        gradients, globNorm = tf.clip_by_global_norm(gradients, 1.0)
        tf.debugging.check_numerics(gradients[0], "GRAD NOT FINITE")
        optimizer.apply_gradients(zip(gradients, postvars))
    return loss


if trainModels:
    
    createNNsPost()
    
    '''Optimization sequence'''
    model.n_sgd      = 1

    num_iterations   = tfint(500_000)

    lr = tf.keras.optimizers.schedules.ExponentialDecay(
        1e-3, int(num_iterations/5), 0.5, staircase=True, name=None
    )
    optimizer = tf.keras.optimizers.Adam( lr )

    elboAll     = []
    startTraining = time.time()
    for epoch in tf.range(0, num_iterations ):

        start_time = time.time()
        loss = train_step_post(optimizer, epoch)

        end_time = time.time()

        if epoch % intervalELBOsave == 0:
            #tf.print('epsi_r = ', model.epsilon_r)
            elboAll.append(-loss)
            print('Epoch: {}, Test set ELBO: {}, time elapse for current epoch: {}'
                .format(epoch, -loss, end_time - start_time))

    endTraining = time.time()
    totalTimeTraining += endTraining - startTraining
    print("DONE TRAINING FOR EPSI", eps)

timeEndTraingPost = time.time()
print('timeTakenTraining = ', timeEndTraingPost - timeStartTraingPost )

In [ ]:
saveModelWeights = False
loadModelWeights = False
'''If you want to load Model Weights'''

if loadModelWeights:
    dirname = 'weightsModel-100/'.format(eps)
    model.postModels['KBlBr_of_Y'].NN.load_weights(dirname+'/KBlBr_of_Y_weights.h5')
    model.postModels['Y_of_U'].NN.load_weights(dirname+'/Y_of_U_weights.h5')

    model.postModels['KBlBr_of_Y'].NN.compile()
    model.postModels['Y_of_U'].NN.compile()
    # Check its architecture
    model.postModels['KBlBr_of_Y'].NN.summary()
    model.postModels['Y_of_U'].NN.summary()
    print("LOADED WEIGHTS")




if saveModelWeights:
    dirname = 'weightsModel-100/'.format(eps)
    os.makedirs(os.path.dirname(dirname), exist_ok=True)
    model.postModels['KBlBr_of_Y'].NN.save_weights(dirname+'/KBlBr_of_Y_weights.h5')
    model.postModels['Y_of_U'].NN.save_weights(dirname+'/Y_of_U_weights.h5')
    print("SAVED WEIGHTS")



In [ ]:
#print(elboAll)
if trainModels:
    elboAll =  np.array(elboAll)
    np.savetxt('ELBOValsModel-{}epsi'.format(eps), elboAll)

print( 'totalTimeTraining = ', totalTimeTraining)
print( 'totalTimeSolving = ', totalTimeSolving)
print( 'totalTimeNNSolving = ', totalTimeNNSolving)


In [ ]:
dataYEpsi =tffloat(0.005)
def missingPhysics(u, x):
    #return u
    return u * (tf.math.sin(x*20.) / 20. + 1.) 
def addIndtNoise(u):
    return u + tf.random.normal(stddev=dataYEpsi, shape=u.shape, dtype=floatf)

def plotTrueSlnSamplesPost(samplesK, samplesF, samplesBcl, samplesBcr, SAVE=None):
    
    fig, ax = plt.subplots(nrows=2, ncols=4, sharex=False,
                                        figsize=(20, 8))
    
    u_All = tf.constant(0., shape=[1, model.dimY], dtype=floatf)

    for i in range(samplesK.shape[0]):
        
        model.pdeState.forcing =  tf.reshape(samplesF[i,:], [-1]) 
        model.pdeState.kappa   =  tf.reshape(samplesK[i,:], [-1]) 
        model.pdeState.bdLeft  =  tf.reshape(samplesBcl[i,:], []) 
        model.pdeState.bdRight =  tf.reshape(samplesBcr[i,:], []) 

        u_prior = solver.solve_tf_bfgs( )
        u_missPhys = missingPhysics(u_prior, xFE)
        u_missPhysNoise = addIndtNoise(u_missPhys)
        u_All = tf.concat([u_All,u_missPhysNoise[None,:] ], 0)

        ax[0,0].plot(pdeState.xFE,u_missPhysNoise)

        ax[0,1].plot(pdeState.xFE, u_prior)
        
        ax[0,2].plot(pdeState.xFE, u_missPhysNoise - u_prior)
        ax[0,2].set_ylim(-0.25, 0.25)

        kappaField = pdeState.kappa_fun(xFE)
        ax[0,3].plot( pdeState.xFE, kappaField )
        
        ZCoeffReconstMean, ZCoeffReconstlogvar = model.postModels['KBlBr_of_Y'].Map([ u_missPhysNoise[None,:]])

        kappaCoeffReconstMean = tf.gather(ZCoeffReconstMean, model.postModels['KBlBr_of_Y'].multOutputLayout['K'], axis=1)
        kappaCoeffReconstlogVar  = tf.gather(ZCoeffReconstlogvar, model.postModels['KBlBr_of_Y'].multOutputLayout['K'], axis=1)

        BLCoeffReconstMean = tf.gather(ZCoeffReconstMean, model.postModels['KBlBr_of_Y'].multOutputLayout['Bl'], axis=1)
        BLCoeffReconstlogVar  = tf.gather(ZCoeffReconstlogvar, model.postModels['KBlBr_of_Y'].multOutputLayout['Bl'], axis=1)

        BRCoeffReconstMean = tf.gather(ZCoeffReconstMean, model.postModels['KBlBr_of_Y'].multOutputLayout['Br'], axis=1)
        BRCoeffReconstlogVar  = tf.gather(ZCoeffReconstlogvar, model.postModels['KBlBr_of_Y'].multOutputLayout['Br'], axis=1)

        kappaCoeffReconstMean =  tf.reshape(kappaCoeffReconstMean,[-1])
        kappaCoeffReconstVar  =  tf.exp(tf.reshape(kappaCoeffReconstlogVar,[-1]))
        meanK, diagCovK = pdeState.gaussChebyGaussCoeffs(kappaCoeffReconstMean,kappaCoeffReconstVar, xFE)
        meanKT, varKT = pdeState.unscentedGaussTransf(meanK, diagCovK, pdeState.kappaBij, k=1)
        std2KT = 2*tf.sqrt(varKT)

        BlReconst = tf.gather(ZCoeffReconstMean, model.postModels['KBlBr_of_Y'].multOutputLayout['Bl'], axis=1)
        BrReconst = tf.gather(ZCoeffReconstMean, model.postModels['KBlBr_of_Y'].multOutputLayout['Br'], axis=1)

        # meanSamplesU, logVarSamplesU = model.priorModels['U_of_KFBlBr'].Map([samplesK[i,:][None,:], samplesF[i,:][None,:], samplesBcl[i,:][None,:], samplesBcr[i,:][None,:]])
        meanSamplesU, logVarSamplesU = model.priorModels['U_of_KFBlBr'].Map([tf.reshape(kappaCoeffReconstMean, [1,-1]), samplesF[i,:][None,:], BLCoeffReconstMean, BRCoeffReconstMean])
        us = model.reparameterize(meanSamplesU, logVarSamplesU )

        meanU, diagVarU = pdeState.gaussChebyGaussCoeffs(meanSamplesU[0,:], tf.exp(logVarSamplesU[0,:]), xFE )
        std2U = 2.*tf.sqrt(diagVarU)

        YReconstMean, YreconLogVar = model.postModels['Y_of_U'].Map([tf.reshape(meanSamplesU[0,:], [1,-1])])  
        std2YT = 2*tf.sqrt(YreconLogVar)

        ax[1,0].plot(pdeState.xFE, YReconstMean[0,:])
        ax[1,0].fill_between(pdeState.xFE, YReconstMean[0,:] - std2YT[0,:], YReconstMean[0,:] + std2YT[0,:], alpha=0.2)

        ax[1,1].plot(pdeState.xFE, meanU)
        ax[1,1].fill_between(pdeState.xFE, meanU - std2U, meanU + std2U, alpha=0.2)

        ax[1,2].plot(pdeState.xFE, YReconstMean[0,:] - meanU)
        ax[1,1].fill_between(pdeState.xFE, (YReconstMean[0,:] - meanU) - (std2U+std2YT[0,:]),  (YReconstMean[0,:] - meanU) + (std2U+std2YT[0,:]), alpha=0.2)
        ax[1,2].set_ylim(-0.25, 0.25)

        ax[1,3].plot(pdeState.xFE, meanKT)
        ax[1,3].fill_between(pdeState.xFE, meanKT - std2KT, meanKT + std2KT, alpha=0.2)

 
    plt.tight_layout()
    if SAVE is not None:
        plt.tight_layout()
        extent = ax[0, 0].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('figs/{}-Datay.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[0, 1].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('figs/{}-TrueSol.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[0, 2].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('figs/{}-TrueRho.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[0, 3].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('figs/{}-TrueKappa.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[1, 0].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('figs/{}-Reconstructedy.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[1, 1].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('figs/{}-ReconstructedSol.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[1, 2].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('figs/{}-ReconstructedRho.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

        extent = ax[1, 3].get_tightbbox(fig.canvas.renderer).transformed(fig.dpi_scale_trans.inverted())
        fig.savefig('figs/{}-ReconstructedKappa.pdf'.format(SAVE), bbox_inches=extent.expanded(1.01, 1.01))

        plt.show()
        plt.savefig('MissingPhysFig{}.pdf'.format(SAVE))
    plt.show()

    u_All = u_All[1:,:]
    return u_All


#model.dataY = plotTrueSlnSamplesPost(samplesKdata, samplesFdata, samplesBldata, samplesBrdata)

In [ ]:
numData = 5

samplesKdataPost   = model.LatentKappaPrior.sample(sample_shape=(numData))
samplesFdataPost   = model.LatentForcingPrior.sample(sample_shape=(numData))
samplesFdataPost   = tf.reshape(tf.linspace(1., 5., 5), [-1, 1])
samplesBldataPost  = model.LatentBlPrior.sample(sample_shape=(numData))
samplesBrdataPost  = model.LatentBrPrior.sample(sample_shape=(numData))


plotTrueSlnSamplesPost(samplesKdataPost, samplesFdataPost, samplesBldataPost, samplesBrdataPost, SAVE=None)